# Item Response Theory with Gradient Descent Optimization (GDIRT)

This notebook will show you how to train and use the GDIRT.
First, we will show how to get the data (here we use a0910 as the dataset).
Then we will show how to train a GDIRT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [IRT.py](IRT.py)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
# Load the data from files
import pandas as pd

train_data = pd.read_csv("../../../data/a0910/train.csv")
valid_data = pd.read_csv("../../../data/a0910/valid.csv")
test_data = pd.read_csv("../../../data/a0910/test.csv")

train_data.head(5)

,user_id,item_id,score
0,1615,12977,1
1,782,13124,0
2,1084,16475,0
3,593,8690,0
4,127,14225,1


In [2]:
len(train_data), len(valid_data), len(test_data)

(186049, 25606, 55760)

In [3]:
# Transform data to torch Dataloader (i.e., batchify)
# batch_size is set to 32

import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 256
def transform(x, y, z, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)

train, valid, test = [
    transform(data["user_id"], data["item_id"], data["score"], batch_size)
    for data in [train_data, valid_data, test_data]
]
train, valid, test

(<torch.utils.data.dataloader.DataLoader at 0x704d507882e0>,
 <torch.utils.data.dataloader.DataLoader at 0x704d50788b20>)

## Training and Persistence

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [7]:
from EduCDM import GDIRT

cdm = GDIRT(4164, 17747)

cdm.train(train, valid, epoch=2)
cdm.save("irt.params")

ModuleNotFoundError: No module named 'EduCDM'

## Loading and Testing

In [6]:
cdm.load("irt.params")
auc, accuracy = cdm.eval(test)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from irt.params
evaluating: 100%|██████████| 218/218 [00:00<00:00, 314.95it/s]


auc: 0.523327, accuracy: 0.610922
